import libraries

In [1]:
import glob
import pandas as pd
import folium
import geopandas as gpd
import sys
from multiprocessing import Pool
from multiprocessing import cpu_count
nCores = cpu_count()
import numpy as np

import project functions

In [65]:
sys.path.append("..")
from src.geoIndexFunctions import *
from src.generalFunctions import *

### GPS data 

In [3]:
gps_path = '../../data/geodata/paul/mixed'

In [4]:
# read all files in dir

all_files = glob.glob(gps_path + "/*.snappy.parquet")

lista = []
cnr = 0
for filename in all_files:
    try:
        cnr +=1
        df = pd.read_parquet(filename, engine='pyarrow')
        lista.append(df)
    except:
        pass

bbox_london = pd.concat(lista, axis=0, ignore_index=True)

In [5]:
# not showing deviceId for data protection reasons
bbox_london.loc[:,bbox_london.columns != 'deviceId'].head()

,timestamp,latitude,longitude,country,startTimestamp,endTimestamp
0,1554138000000,51.422662,-0.204717,GBR,1554141385310,1554141389443
1,1554138000000,51.472333,-0.487267,GBR,1554140558907,1554141310298
2,1554138000000,51.477352,-0.112836,GBR,1554139152516,1554139296345
3,1554138000000,51.462253,-0.170607,GBR,1554138718963,1554138718963
4,1554138000000,51.516440,-0.131118,GBR,1554139984927,1554139984927


In [6]:
bbox_london['latitude'] = bbox_london['latitude'].astype('float')
bbox_london['longitude'] = bbox_london['longitude'].astype('float')

In [7]:
bbox_london['country'].value_counts().head(10)

GBR    7554970
USA     201756
FRA     148919
ESP     117627
ITA     115004
DEU      68988
NLD      61924
PHL      45330
AUS      30872
SWE      27601
Name: country, dtype: int64

In [8]:
def size_devices_df(df, df_name):
    print(df_name)
    print("Size of dataframe: ",len(df))
    # how many unique device ids in df
    print("Unique device ids: ", len(df.deviceId.unique().tolist()))
    # how many unique countries in df
    print("Unique countries: ", df.country.nunique())

In [9]:
size_devices_df(bbox_london, 'bbox_london')

bbox_london
Size of dataframe:  8589632
Unique device ids:  1071401
Unique countries:  87


## Smart Pin 

Speak about the checks that are getting applied

### Select only the events within the GADM boundary of London 

computationally expensive

In [10]:
# path = '/home/lefteris/Desktop/trajectories/data/GADM/gadm36_GBR_shp/gadm36_GBR_2.shp'
# city = 'London'

# l_gdf = get_city_from_gadm(city, path)

# l_gdf.plot()

add geometry to bbox_london

In [11]:
# from shapely import geometry
# from shapely.geometry import Point

# # split the dataframe into chunks for parallelization
# bbox_london_split = np.array_split(bbox_london, nCores)

# def add_point_geom(df):
#     df['geometry'] = df.apply(lambda x: geometry.Point(x.longitude, x.latitude), axis=1)
#     return df

# # multi cores with Multiprocessing
# with Pool(processes=nCores) as pool1:
#     point_bbox_london = pd.concat(pool1.map(add_point_geom, bbox_london_split)).reset_index(drop=True)

sjoin with geopandas to get only the points i.e. gps events within London

### Plot devices on folium 

In [82]:
c_bbox_london = bbox_london.copy()

In [83]:
# filter by country???
country_code = 'ITA'

In [84]:
c_bbox_london = c_bbox_london[(c_bbox_london.deviceId!='') & (c_bbox_london.country==country_code)]

In [85]:
# not showing deviceId for data protection reasons
c_bbox_london.loc[:,c_bbox_london.columns != 'deviceId'].head()

,timestamp,latitude,longitude,country,startTimestamp,endTimestamp
3777472,1554822000000,51.493272,-0.172081,ITA,1554824989084,1554825510055
3777480,1554922800000,51.525055,-0.125476,ITA,1554925101119,1554925101424
3777484,1554922800000,51.517487,-0.080943,ITA,1554925941072,1554925950760
3777498,1555416000000,51.499603,-0.198635,ITA,1555419271666,1555419272189
3777508,1555707600000,51.535314,-0.139244,ITA,1555709645200,1555709648518


In [86]:
# sort by starting time
c_bbox_london.sort_values(by=['startTimestamp'], ascending=True, inplace=True)

# set the starting time as the df index
c_bbox_london.set_index('startTimestamp',drop=False, inplace=True)

# drop timestamp column
c_bbox_london = c_bbox_london[["deviceId","latitude","longitude"]]

In [87]:
# select the top 10 ids
top_ids = c_bbox_london['deviceId'].value_counts().index.values.tolist()[:10]

In [88]:
# c_bbox_london[c_bbox_london['deviceId']==top_ids[1]]

In [95]:
from shapely import geometry
from h3 import h3

APERTURE_SIZE = 9
hex_col = 'hex'+str(APERTURE_SIZE)

# find hexs containing the points
c_bbox_london[hex_col] = c_bbox_london.apply(lambda x: h3.geo_to_h3(x.latitude,x.longitude,APERTURE_SIZE),axis=1)
# df_noDupl_1dev['geometry'] = df_noDupl_1dev.apply(lambda x:geometry.Polygon(h3.h3_to_geo_boundary(x[hex_col], geo_json=True)),axis=1)

In [103]:
hex_list = c_bbox_london[hex_col].unique().tolist()
len(hex_list)

6172

In [94]:
import folium
map_dev = folium.Map(location=[51.493371, -0.130066],
                    zoom_start = 10)

for id_value in top_ids:
    
    df_noDupl_1dev = c_bbox_london[c_bbox_london['deviceId']==id_value]
    trajectory_points = df_noDupl_1dev.sort_values(by='startTimestamp')[["latitude","longitude"]].values #.as_matrix(columns=["latitude","longitude"])

    folium.PolyLine(trajectory_points, color="red", weight=2.5, opacity=1).add_to(map_dev)

map_dev

In [115]:
# we cannot visualize everythin
visualize_hexagons(hex_list[:1000])

In [116]:
visualize_hexagons(h3.compact(hex_list[:2000]))